In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
eve = input()
url = "https://www.vlr.gg/" + eve
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

- Avoir les matchs id = div -> class = 'vm-stats-game' puis data-game-id = 'all' ou 'xxxxxx'
- 'data-game-id' à récupérer => "vlr.gg/eve/?game='data-game-id'"
- &tab=overview/performance/economy
- Stats dans 'vm-stats-container'

Partie overview
- All maps.
- Then match per match.

All maps

In [3]:
all_maps = soup.find_all("div", {"class": "vm-stats-game"})
game_ids = []
for maps in all_maps:
    game_id = maps.get('data-game-id')
    game_ids.append(game_id)
maps

<div class="vm-stats-game" data-game-id="152028">
<div class="vm-stats-game-header">
<div class="team">
<div class="score" style="margin-right: 12px;">10 </div>
<div>
<div class="team-name">
							CGN Esports						</div>
<span class="mod-ct">7</span> /
						<span class="mod-t">3</span>
</div>
</div>
<div class="map">
<div style="font-weight: 700; font-size: 20px; text-align: center; position: relative; margin-bottom: 3px;">
<span style="position: relative;">
							Sunset														<span class="picked mod-1 ge-text-light">
								PICK
							</span>
</span>
</div>
<div class="map-duration ge-text-light" style="text-align: center;">
						
													52:56												
						
					

					</div>
</div>
<div class="team mod-right">
<div>
<div class="team-name">
							Acend						</div>
<span class="mod-t">5</span> /
						<span class="mod-ct">8</span>
</div>
<div class="score mod-win" style="margin-left: 8px;">13</div>
</div>
</div>
<div style="text-align: center; margin-top: 15px;

Dans map, deux élements par team.

In [4]:
for e in maps.find_all("table", class_='wf-table-inset mod-overview'):
    team_1 = maps.find_all("table", class_='wf-table-inset mod-overview')[0]
    team_2 = maps.find_all("table", class_='wf-table-inset mod-overview')[1]

In [5]:
stats_t1 = []
header_row = []
t_n = []
p_n = []
champions = []
num_columns = 12

for el in team_1.find_all('span', class_='mod-both'):
    stats_t1.append(el.get_text(strip=True))
table_data = [stats_t1[i:i + num_columns] for i in range(0, len(stats_t1), num_columns)]

for title in team_1.find_all('th'):
    header_row.append(title.get('title'))
header_row = list(filter(None, header_row))
header_row.remove('Agent')
for el in team_1.find_all('div', class_='ge-text-light'):
    t_n.append(el.get_text(strip=True))
for el in team_1.find_all('div', class_='text-of'):
    p_n.append(el.get_text(strip=True))
for el in team_1.find_all('span', class_='mod-agent'):
    champions.append(el.find('img').get('title'))
    
df_data = pd.DataFrame(table_data, columns=header_row)
df_champions = pd.DataFrame(champions, columns=['Agents'])
df_t_n = pd.DataFrame(t_n,columns=['Team'])
df_p_n = pd.DataFrame(p_n,columns=['Player'])
df = pd.concat([df_t_n, df_p_n, df_champions, df_data], axis=1)

In [6]:
stats_t2 = []
header_row = []
t_n = []
p_n = []
champions = []
num_columns = 12

for el in team_2.find_all('span', class_='mod-both'):
    stats_t2.append(el.get_text(strip=True))
table_data = [stats_t2[i:i + num_columns] for i in range(0, len(stats_t2), num_columns)]

for title in team_2.find_all('th'):
    header_row.append(title.get('title'))
header_row = list(filter(None, header_row))
header_row.remove('Agent')
for el in team_2.find_all('div', class_='ge-text-light'):
    t_n.append(el.get_text(strip=True))
for el in team_2.find_all('div', class_='text-of'):
    p_n.append(el.get_text(strip=True))
for el in team_2.find_all('span', class_='mod-agent'):
    champions.append(el.find('img').get('title'))
    
df_data = pd.DataFrame(table_data, columns=header_row)
df_champions = pd.DataFrame(champions, columns=['Agents'])
df_t_n = pd.DataFrame(t_n,columns=['Team'])
df_p_n = pd.DataFrame(p_n,columns=['Player'])
df2 = pd.concat([df_t_n, df_p_n, df_champions, df_data], axis=1)
df = pd.concat([df, df2], axis=0, ignore_index=True)
df = df.dropna()
df

,Team,Player,Agents,Rating,Average Combat Score,Kills,Deaths,Assists,Kills - Deaths,"Kill, Assist, Trade, Survive %",Average Damage per Round,Headshot %,First Kills,First Deaths,Kills - Deaths
0,CGN,Vince,Omen,1.21,257,20,16,8,+4,74%,159,33%,2,1,+1
1,CGN,BennY,Phoenix,1.17,231,21,16,2,+5,78%,154,34%,4,4,0
2,CGN,DaviH,Skye,1.03,151,12,14,6,-2,78%,107,32%,1,2,-1
3,CGN,aNguiSt,Cypher,0.74,151,13,17,3,-4,65%,112,64%,2,1,+1
4,CGN,elllement,Raze,0.58,124,11,20,2,-9,57%,84,29%,2,4,-2
5,ACE,chiwa,Cypher,1.07,171,13,12,8,+1,78%,117,20%,0,0,0
6,ACE,baddyG,Viper,1.05,250,20,17,5,+3,74%,168,38%,4,3,+1
7,ACE,musashi,Gekko,1.00,215,18,16,3,+2,65%,136,27%,2,1,+1
8,ACE,ALIVE,Raze,0.98,219,19,17,2,+2,70%,141,17%,4,5,-1
9,ACE,Br0die,Omen,0.96,168,13,15,6,-2,87%,115,26%,2,2,0


Partie Performance

In [7]:
eve = '291468'
url = "https://www.vlr.gg/" + eve + "?tab=performance"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
all_maps = soup.find_all("div", {"class": "vm-stats-game"})

In [8]:
perf = all_maps[0].find_all("table", {"class": "wf-table-inset mod-matrix mod-normal"})[0]

In [9]:
perf_indi = []
player_header = []
team_header = []
comp = []
num_columns = 15
for els in perf.find_all('div', {'class': 'team'}):
    player_header.append(els.find('div').contents[0].strip())
    team_header.append(els.find('div', class_='team-tag').contents[0].strip())
for el in perf.find_all("div", {"class": "stats-sq"}):
    perf_indi.append(el.get_text(strip=True))
for i in range(len(player_header)):
    comp.append(team_header[i] + ' ' + player_header[i])

table_data = [perf_indi[i:i + num_columns] for i in range(0, len(perf_indi), num_columns)]
df = pd.DataFrame(data=table_data)
non_header = [1,2,4,5,7,8,10,11,13,14]
for a in range(len(non_header)):
    df.rename(columns={df.columns[non_header[a]]:""}, inplace=True)
#df.columns = [nouveau_nom if any(char.isdigit() for char in str(col)) else col for col, nouveau_nom in zip(df.columns, comp[0:5])]
df.rename(columns={df.columns[0]:comp[0],
                   df.columns[3]:comp[1],
                   df.columns[6]:comp[2],
                   df.columns[9]:comp[3],
                   df.columns[12]:comp[4]}, inplace=True)
index = pd.DataFrame(comp[5:],columns=['Player'])
df = pd.concat([index,df], axis=1)
df.set_index(['Player'])

,ACE musashi,,,ACE Br0die,,,ACE ALIVE,,,ACE baddyG,,,ACE chiwa,,
Player,,,,,,,,,,,,,,,
CGN BennY,8,9,-1,8,6,+2,7,12,-5,8,11,-3,5,10,-5
CGN Vince,8,12,-4,12,5,+7,9,6,+3,13,13,+0,8,8,+0
CGN elllement,9,8,+1,7,12,-5,10,10,+0,2,8,-6,8,11,-3
CGN DaviH,7,12,-5,5,9,-4,6,8,-2,11,9,+2,4,6,-2
CGN aNguiSt,9,7,+2,5,6,-1,10,13,-3,6,6,+0,7,11,-4


In [10]:
perf = all_maps[1].find_all("table", {"class": "wf-table-inset mod-adv-stats"})[0]
perf

<table class="wf-table-inset mod-adv-stats">
<tr>
<th></th>
<th></th>
<th>2K</th>
<th>3K</th>
<th>4K</th>
<th>5K</th>
<th>1v1</th>
<th>1v2</th>
<th>1v3</th>
<th>1v4</th>
<th>1v5</th>
<th>ECON</th>
<th>PL</th>
<th>DE</th>
</tr>
<tr>
<td>
<div class="team">
<img class="team-logo" src="//owcdn.net/img/60defafec983c.png"/>
<div>
																			elllement																		<div class="team-tag ge-text-faded">
										CGN									</div>
</div>
</div>
</td>
<td>
<div class="stats-sq">
<img src="/img/vlr/game/agents/jett.png" style="width: 28px;"/>
</div>
</td>
<td>
<div class="stats-sq wf-popable vm-perf-notable mod-d">
																			3									
									
									<div class="wf-popable-contents vm-perf-notable-scope">
<div style="margin-top: -10px;">
<div style="margin-top: 10px;">
<div style="white-space: nowrap;">
														Round <span>5</span>
</div>
<div style="display: flex; align-items: center; margin-top: 5px;">
<img src="/img/vlr/game/agents/omen.png"/> Br0die												

In [11]:
perf_indi = []
header = []
player = []
team = []
comp = []
num_columns = 13
for els in perf.find_all('th'):
    header.append(els.get_text(strip=True))
for els in perf.find_all('div', {'class': 'team'}):
    player.append(els.find('div').contents[0].strip())
    team.append(els.find('div', class_='team-tag').contents[0].strip())
for i in range(len(player)):
    comp.append(team[i] + ' ' + player[i])
df_comp = pd.DataFrame(comp, columns=['Player'])
for el in perf.find_all("div", {"class": "stats-sq"}):
    perf_indi.append(el.contents[0].strip())
table_data = [perf_indi[i:i + num_columns] for i in range(0, len(perf_indi), num_columns)]

df = pd.DataFrame(data=table_data, columns=header[1:])
df = pd.concat([df_comp, df], axis=1)
df = df.drop(columns=df.columns[1])
df.set_index('Player')

,2K,3K,4K,5K,1v1,1v2,1v3,1v4,1v5,ECON,PL,DE
Player,,,,,,,,,,,,
CGN elllement,3,,,,,,,,,44,1,1
CGN BennY,2,,,,,,,,,37,1,0
CGN Vince,1,1,1,,1,,,,,63,2,0
CGN aNguiSt,4,1,,,,,,,,73,2,0
CGN DaviH,1,,,,,,,,,34,0,1
ACE chiwa,2,2,,,1,,,,,67,1,1
ACE ALIVE,4,,,,1,,,,,39,0,1
ACE baddyG,3,,,,,,,,,51,1,0
ACE Br0die,3,1,,,1,,,,,53,6,1


Partie Economie

In [12]:
eve = '291468'
url = "https://www.vlr.gg/" + eve + "?tab=economy"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
all_maps = soup.find_all("div", {"class": "vm-stats-game"})
perf = all_maps[0].find_all("table", {"class": "wf-table-inset mod-econ"})
eco_overall = perf[0]
eco_round = perf[1]
perf

[<table class="wf-table-inset mod-econ">
 <tr>
 <th></th>
 <th>
 								Pistol Won				
 
 							</th>
 <th>
 								Eco (won)				
 
 							</th>
 <th>
 								$ (won)				
 
 							</th>
 <th>
 								$$ (won)				
 
 							</th>
 <th>
 								$$$ (won)				
 
 							</th>
 </tr>
 <tr>
 <td>
 <div class="team">
 <img src="//owcdn.net/img/60defafec983c.png"/>
 										
 									
 																		CGN								</div>
 </td>
 <td>
 <div class="stats-sq">
 																					0																			</div>
 </td>
 <td>
 <div class="stats-sq">
 																					3											(0)
 																			</div>
 </td>
 <td>
 <div class="stats-sq">
 																					1											(0)
 																			</div>
 </td>
 <td>
 <div class="stats-sq">
 																					6											(4)
 																			</div>
 </td>
 <td>
 <div class="stats-sq">
 																					11											(4)
 																			</div>
 </td>
 </tr>
 <tr>
 <td>
 <div class="team">
 <img src="//owcdn.net/img/62a4

In [13]:

header = []
for els in eco_overall.find_all('th'):
    header.append(els.get_text(strip=True))
header.remove('')
team = []
for els in eco_overall.find_all('div', {'class': 'team'}):
    team.append(els.get_text(strip=True))
eco = []
for el in eco_overall.find_all("div", {"class": "stats-sq"}):
    eco.append(el.get_text(strip=True).replace('\t\t', '').replace('\t', ' '))
df_eco = pd.DataFrame(columns=header)
df_eco.loc[0] = eco[0:5]
df_eco.loc[1] = eco[5:10]
df = pd.DataFrame(data=team, columns=['Team'])
df = pd.concat([df, df_eco], axis=1)
df.set_index(['Team'], inplace=True)
df

,Pistol Won,Eco (won),$ (won),$$ (won),$$$ (won)
Team,,,,,
CGN,0,3 (0),1 (0),6 (4),11 (4)
ACE,2,2 (2),2 (0),8 (4),9 (7)


In [16]:
header = []
for els in range(len(eco_round.find_all('div', class_="round-num"))):
    header.append(els+1)
bank = []
for el in eco_round.find_all('div', class_='bank'):
    bank.append(el.get_text(strip=True))
sq = []
for el in eco_round.find_all('div', class_='rnd-sq'):
    sq.append(el.get_text(strip=True))
team = ['Bank1']
for els in eco_overall.find_all('div', {'class': 'team'}):
    team.append(els.get_text(strip=True))
team.append('Bank2')
num_columns = len(header)
df_banks = pd.DataFrame(bank, columns=['Bank'])
df_sq = pd.DataFrame(sq, columns=["SQ"])
df3 = pd.DataFrame(index=team,columns=header)
for i, col in enumerate(df3.columns):
    df3.at['Bank1', col] = df_banks['Bank'][2 * i]
    df3.at['Bank2', col] = df_banks['Bank'][2 * i + 1]
for i, col in enumerate(df3.columns):
    df3.at[df3.index[1], col] = df_sq['SQ'][2 * i]
    df3.at[df3.index[2], col] = df_sq['SQ'][2 * i + 1]    
df3

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
Bank1,0.3k,6.9k,0.9k,7.7k,18.1k,24.9k,17.9k,19.7k,12.1k,5.3k,...,9.0k,0.2k,7.9k,2.8k,14.1k,13.9k,16.2k,5.1k,2.0k,0.5k
CGN,,,$$,$$,$$$,$$$,$$$,$$$,$$$,$$$,...,$$$,,$,$$$,$$,$$$,$$$,$$$,$$,$$
ACE,,$$,$$$,$$,$,$$,$$,$$,$$$,$$$,...,$$$,,$$,$$,$$,$,$$$,$$$,$$$,$$$
Bank2,0.1k,5.6k,14.1k,5.5k,9.8k,7.4k,15.6k,11.1k,15.1k,18.4k,...,10.3k,0.4k,0.7k,5.6k,3.4k,11.1k,5.9k,19.5k,17.6k,19.3k
